<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [3]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [4]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,-1403073183891835564,351299
1,3545707052733304728,327823
2,1451354159195218418,334485
3,-7773938200482214258,343795
4,-328012383083104805,359891


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [5]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 15083 examples in the train dataset and 1511 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [6]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.988654,True,13,1,38.0,-9068386407968572094
1,6.812284,True,13,1,40.0,-1922480672677032316
2,6.444112,True,13,1,39.0,-5937540421097454372
3,6.563162,True,13,1,34.0,7895641210289919997
4,3.875727,False,14,1,31.0,1011945037241428753


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [7]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,15069.000000,15083.000000,15083.000000,14975.000000,1.508300e+04
mean,7.222424,27.328781,1.033017,38.612821,-3.138648e+17
std,1.310411,6.178642,0.190191,2.545370,5.227885e+18
min,0.500449,13.000000,1.000000,17.000000,-9.183606e+18
25%,6.563162,22.000000,1.000000,38.000000,-4.979698e+18
50%,7.312733,27.000000,1.000000,39.000000,-1.002950e+18
75%,8.000575,32.000000,1.000000,40.000000,3.572456e+18
max,13.999354,49.000000,4.000000,47.000000,8.599690e+18


In [10]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [11]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.988654,True,13,Single(1),38.0,-9068386407968572094
1,6.812284,True,13,Single(1),40.0,-1922480672677032316
2,6.444112,True,13,Single(1),39.0,-5937540421097454372
3,6.563162,True,13,Single(1),34.0,7895641210289919997
4,3.875727,False,14,Single(1),31.0,1011945037241428753


In [12]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
15077,8.562754,Unknown,47,Single(1),39.0,-4701948054522374613
15078,9.735613,Unknown,47,Single(1),39.0,124458947937444850
15080,9.539402,Unknown,49,Single(1),39.0,8599690069971956834
15081,3.249614,Unknown,49,Multiple(2+),31.0,1148502204931914436
15082,4.750962,Unknown,49,Single(1),36.0,7895641210289919997


In [13]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,29928.000000,29928.000000,29928.000000,2.992800e+04
mean,7.224877,27.326918,38.617749,-3.091691e+17
std,1.308462,6.173437,2.529335,5.225591e+18
min,0.500449,13.000000,17.000000,-9.183606e+18
25%,6.563162,22.000000,38.000000,-4.979698e+18
50%,7.312733,27.000000,39.000000,-1.002950e+18
75%,8.000575,32.000000,40.000000,3.572456e+18
max,13.999354,49.000000,47.000000,8.599690e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [14]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [15]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   3006 eval.csv
  29928 train.csv
  32934 total
==> eval.csv <==
5.40573466424,True,14,Single(1),38.0,1088037545023002395
5.6879263596,False,16,Single(1),34.0,1088037545023002395
7.936641432,False,16,Single(1),41.0,1088037545023002395
4.49963476742,True,16,Single(1),37.0,1088037545023002395
5.99877814902,False,17,Single(1),40.0,1088037545023002395
8.24969784404,False,17,Single(1),39.0,1088037545023002395
6.3889963527599996,False,18,Single(1),38.0,1088037545023002395
7.50012615324,False,18,Single(1),40.0,1088037545023002395
6.2611282408,False,18,Single(1),40.0,1088037545023002395
6.4374980503999994,True,18,Single(1),37.0,1088037545023002395

==> train.csv <==
6.9886537054,True,13,Single(1),38.0,-9068386407968572094
6.8122838958,True,13,Single(1),40.0,-1922480672677032316
6.44411191826,True,13,Single(1),39.0,-5937540421097454372
6.56316153974,True,13,Single(1),34.0,7895641210289919997
3.87572656596,False,14,Single(1),31.0,1011945037241428753
5.7981574905999995,True,14,Single(1),36.0,-60

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License